In [85]:
# Required Libraries
import pandas as pd
import numpy as np
import pm4py
import sklearn
from datetime import timedelta
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [76]:
# reading in the xes file, make sure it is located in the directory
log = pm4py.read_xes(r"C:\Users\Adaml\OneDrive\Desktop\BPI Challenge 2017.xes") 
df = pm4py.convert_to_dataframe(log) 
df['time:timestamp']= pd.to_datetime(df['time:timestamp']).dt.tz_convert(None) # Removing the UTC from timestamp

parsing log, completed traces :: 100%|██████████| 31509/31509 [01:32<00:00, 340.74it/s]


In [77]:
#TimeStamp to int
df['startYear'] = df['time:timestamp'].apply(lambda time: time.year)
df['startMonth'] = df['time:timestamp'].apply(lambda time: time.month)
df['startDay'] = df['time:timestamp'].apply(lambda time: time.day)
df['startHour'] = df['time:timestamp'].apply(lambda time: time.hour)
df['startMinute'] = df['time:timestamp'].apply(lambda time: time.minute)
df['startSecond'] = df['time:timestamp'].apply(lambda time: time.second)
df = df.drop('time:timestamp', axis=1)

In [78]:
#Assigning Next Event
df['next_event'] = df['concept:name'].shift(-1)

In [79]:
d_last = df.index.to_series().groupby(df['org:resource']).agg(['last']).reset_index() # Get indexes of last event in each case
last_lst = d_last['last'].to_list() # Indexes of last event to list

for num in last_lst:
    df.loc[num, 'next_event'] = np.NaN # Replace last events in each case to NaN

df

,Action,org:resource,concept:name,EventOrigin,EventID,lifecycle:transition,case:LoanGoal,case:ApplicationType,case:concept:name,case:RequestedAmount,...,CreditScore,OfferedAmount,OfferID,startYear,startMonth,startDay,startHour,startMinute,startSecond,next_event
0,Created,User_1,A_Create Application,Application,Application_652823628,complete,Existing loan takeover,New credit,Application_652823628,20000.0,...,NaN,NaN,NaN,2016,1,1,9,51,15,A_Submitted
1,statechange,User_1,A_Submitted,Application,ApplState_1582051990,complete,Existing loan takeover,New credit,Application_652823628,20000.0,...,NaN,NaN,NaN,2016,1,1,9,51,15,W_Handle leads
2,Created,User_1,W_Handle leads,Workflow,Workitem_1298499574,schedule,Existing loan takeover,New credit,Application_652823628,20000.0,...,NaN,NaN,NaN,2016,1,1,9,51,15,W_Handle leads
3,Deleted,User_1,W_Handle leads,Workflow,Workitem_1673366067,withdraw,Existing loan takeover,New credit,Application_652823628,20000.0,...,NaN,NaN,NaN,2016,1,1,9,52,36,W_Complete application
4,Created,User_1,W_Complete application,Workflow,Workitem_1493664571,schedule,Existing loan takeover,New credit,Application_652823628,20000.0,...,NaN,NaN,NaN,2016,1,1,9,52,36,A_Concept
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1202262,Deleted,User_1,W_Call after offers,Workflow,Workitem_1817549786,ate_abort,Home improvement,New credit,Application_1350494635,20000.0,...,NaN,NaN,NaN,2017,1,6,6,33,2,W_Call after offers
1202263,Created,User_1,W_Call after offers,Workflow,Workitem_363876066,schedule,Home improvement,New credit,Application_1350494635,20000.0,...,NaN,NaN,NaN,2017,1,6,6,33,2,NaN
1202264,statechange,User_28,A_Cancelled,Application,ApplState_1869071797,complete,Home improvement,New credit,Application_1350494635,20000.0,...,NaN,NaN,NaN,2017,1,16,9,51,21,O_Cancelled
1202265,statechange,User_28,O_Cancelled,Offer,OfferState_420066181,complete,Home improvement,New credit,Application_1350494635,20000.0,...,NaN,NaN,Offer_1580299144,2017,1,16,9,51,21,W_Call after offers


In [83]:
# Changing all variables so they have a numeric representation
# Preparation for hot Encoding

# d = {ni: indi for indi, ni in enumerate(set(df['concept:name']))}
# numbers = [d[ni] for ni in df['concept:name']]
# numbers

events = df['concept:name'].factorize()
df['concept:name'] = events[0]

EventOrigin = df['EventOrigin'].factorize()
df['EventOrigin'] = EventOrigin[0]

Action = df['Action'].factorize()
df['Action'] = Action[0]

LoanGoal = df['case:LoanGoal'].factorize()
df['case:LoanGoal'] = LoanGoal[0]

appType = df['case:ApplicationType'].factorize()
df['case:ApplicationType'] = appType[0]

next = df['next_event'].factorize()
df['next_event'] = next[0]

df

,Action,org:resource,concept:name,EventOrigin,EventID,lifecycle:transition,case:LoanGoal,case:ApplicationType,case:concept:name,case:RequestedAmount,...,CreditScore,OfferedAmount,OfferID,startYear,startMonth,startDay,startHour,startMinute,startSecond,next_event
0,0,User_1,0,0,Application_652823628,complete,0,0,0,20000.0,...,NaN,NaN,NaN,2016,1,1,9,51,15,0
1,1,User_1,1,0,ApplState_1582051990,complete,0,0,0,20000.0,...,NaN,NaN,NaN,2016,1,1,9,51,15,1
2,0,User_1,2,1,Workitem_1298499574,schedule,0,0,0,20000.0,...,NaN,NaN,NaN,2016,1,1,9,51,15,1
3,2,User_1,2,1,Workitem_1673366067,withdraw,0,0,0,20000.0,...,NaN,NaN,NaN,2016,1,1,9,52,36,2
4,0,User_1,3,1,Workitem_1493664571,schedule,0,0,0,20000.0,...,NaN,NaN,NaN,2016,1,1,9,52,36,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1202262,2,User_1,9,1,Workitem_1817549786,ate_abort,1,0,31508,20000.0,...,NaN,NaN,NaN,2017,1,6,6,33,2,8
1202263,0,User_1,9,1,Workitem_363876066,schedule,1,0,31508,20000.0,...,NaN,NaN,NaN,2017,1,6,6,33,2,-1
1202264,1,User_28,21,0,ApplState_1869071797,complete,1,0,31508,20000.0,...,NaN,NaN,NaN,2017,1,16,9,51,21,20
1202265,1,User_28,20,2,OfferState_420066181,complete,1,0,31508,20000.0,...,NaN,NaN,Offer_1580299144,2017,1,16,9,51,21,8


In [84]:
# Getting hot encoding because, using 50000 because it overloads the system otherwise
train = df.loc[:10000]
hot = pd.get_dummies(train)
hot= hot.drop('concept:name', axis = 1) #To seperate Features from Labels
hot= hot.drop('next_event', axis = 1) 

# Labels are the values we want to predict
# Features are the values we want to use to predict
# Saving feature names for later use

features = hot.values
features

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 2., 0., ..., 0., 0., 0.],
       [1., 2., 0., ..., 0., 0., 0.],
       [1., 2., 0., ..., 0., 0., 0.]])

In [ ]:
# Training and Testing Sets
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.2,
                                                                           random_state = 42)

In [ ]:
# Instantiate model 
rf = RandomForestRegressor(n_estimators= 1000, random_state=42)
# Train the model on training data
rf.fit(train_features, train_labels)